In [1]:
%display latex

In [165]:
import numpy as np
from sage.all import *
from mubs_utils import testMubs, saveMubs

First we define all the necessary data. We are working with five qubits, therefore $\mathcal H = \mathbb (C^{2})^{\otimes n}$. We need a field of 32 elements, $F$, which we will fix the order by powers of a generating element.

In [301]:
p = 2
n = 5
d = p**n

In [167]:
F = GF(d, 'a')
a = F.gen()
F = [F(0)] + [a**i for i in range(d-1)]

In order to solve Sainz's phase equation or to construct the eigenvectors following Kanat, we must do the pertinent calculations in the Galois ring $GR(4,5)$ of 1,024 elements. To do this we find an irreducible polynomial over the ring of integers modulo 4, then take the quotient:

In [168]:
R = PolynomialRing(Integers(4), 't'); t = R.gen()

The irreducible polynomial chosen is:

In [169]:
factor(t**5 + t**2 + 1)

t^5 + t^2 + 1

In [170]:
GR = R.quotient(t**5 + t**2 + 1, 'x'); x = GR.gen()
GR

Univariate Quotient Polynomial Ring in x over Ring of integers modulo 4 with modulus t^5 + t^2 + 1

The element $x$ is a generator of the Galois ring which is given by a root of the polynomial:

In [171]:
x**5 + x**2 + 1 # verify that the poly does factor

0

It is important to note the ring generator does not generate the multiplicative cyclic subgroup of $GR(4,5)$, as can be seen by taking its $31$st power:

In [172]:
# the ring generator is NOT the multiplicative group generator
x**(2**5-1)

2*x^3 + 2*x + 1

Dr. Andres found the generator we are looking for:

In [173]:
# Don't know how but this works.
xi = x**2 + 2*x + 1
xi**(2**5-1)

1

We construct the Teichmüller $\mathcal T$ as the union of $\{0\}$ and a cyclic subgroup of $GR(4,5)$ of size $2^5-1$ which is isomorphic to $F^*$. We can obtain this set using the generator $\xi$:

In [174]:
T = [GR(0)] + [xi**k for k in range(2**n - 2 + 1)]
len(T)

32

Every element of the Teichmüller set can be given a $2$-adic representation, i.e., for $x \in \mathcal T$ we can find elements $a,b \in \mathcal T$ such that $x = a + 2b$. 

The ring trace is then defined as
$$Tr(x) = a + a^2 + ... + a^{2^{r-1}} + 2(b + b^2 + ... + b^{2^{r-1}})$$
for $x = a + 2b$ and $r = 5$.

The set $\mathcal T$ is isomorphic to the finite field $F$. The mapping the achieves this is called the Teichmüller lift and we assume that it is given by the power order, that is to say the generator of $F$ is mapped to the generator of $\mathcal T$. In this case
$$a \mapsto \xi = x^2 + 2x + 1,$$
where $a$ is the generator of the field $F$ and $x$ is the generator of the ring.

In [175]:
def ringTrace(a, b):
    s = GR(0)
    for k in range(n):
        s += a**(2**k) + GR(2)*b**(2**k)
    return s 

In [176]:
# To use the ring trace as defined previously we
# must decompose x into its 2-adic representation.
def brute2adic(x):
    for a in T:
        for b in T:
            if x == a + GR(2)*b:
                return (a,b)

# This one is slightly faster
def dumb2adic(x):
    for b in T:
        a = x - GR(2)*b
        if a in T:
            return (a,b)

Now let's get in the phase space construction. First of all we have to setup the generators of the Pauli group:
$$
Z_\alpha = \sum_{\kappa} \chi(\kappa \alpha) |\kappa\rangle \langle\kappa|,
\quad
X_\beta  = \sum_{\kappa} |\kappa + \beta\rangle \langle \kappa|.
$$

Here we see our first parting point in methodology. The physicists (Vourdas, Klimov) have used the ordering $Z_\alpha X_\beta$ whereas the mathematicians have chosen $X_\alpha Z_\beta$ (Kanat, Kantor). Since these operators don't commute we have to take into account the ordering even though we know that:
$$
Z_\alpha X_\beta = \chi(\alpha\beta) X_\beta Z_\alpha,
$$
especially since we consider the phase space to be a grid $F \oplus F$, where the points $(\alpha,\beta)$ label the operators. With the first ordering, the operators corresponding to the horizontal ray $(\alpha, 0)$ are the $Z_\alpha$ operators, whereas for the second ordering the ray corresponds to the $X_\alpha$ operators. For the first ordering the computational basis will be assigned to the horizontal lines, while the Fourier basis will be vertical. This works well with the rotation operators viewpoint of the curves in phase space. As all the regular curves can be obtained by "rotation" operators from the horizontal curves, in other words there exists $V_{f}$ such that $V_{f}Z_\alpha V_{f}^\dagger = Z_\alpha X_{f(\alpha)}$, ignoring phase.

The additive character $\chi(\alpha) = \omega^{\alpha}$ where $\omega = e^{2\pi i / p}$ is primitive $p$-root of the $\mathbb C$. In this case ($p = 2$) we have:

In [177]:
exp(2*pi*I/2)

-1

I'm going to define the displacement operators as Sainz does, that means we will use the physicists order:
$$D(\alpha,\beta) = \Phi(\alpha,\beta) Z(\alpha) X(\beta).$$

In [178]:
def Chi(x):
    return exp(2*pi*I/2 * Integer(x.trace()))

In [423]:
# Define the Fourier operator
FF = zero_matrix(SR, d, d)
for i, m in enumerate(F):
    for j, n in enumerate(F):
        FF[i,j] = Chi(m * n) / sqrt(d)

In [185]:
# The Z operator is diagonal in the standard basis
def Z(a):
    return diagonal_matrix([Chi(a * b) for b in F])

In [186]:
# The X operator is just a permutation matrix
def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

In [194]:
def D(a,b):
    phase = 1
    return phase * Z(a) * X(b)

In [195]:
D(F[3], F[4]) * D(F[3], F[4]).conjugate_transpose() == identity_matrix(d)

True

In [201]:
curves = [lambda t: a**i * t for i in range(d)]
len(curves)

32

In [209]:
ray = [(t, curves[12](t)) for t in F]

In [217]:
# check commutativity
for p1 in ray:
    for p2 in ray:
        if D(*p1) * D(*p2) != D(*p2) * D(*p1):
            raise Exception('Operators do not commute!')

At least for $p = 2$, it seems that the operators all commute using the standard rays, nothing seems affected by ignoring phases. What would happen if we added the corresponding phase?

Well, since $Z(\alpha)X(\beta)$ and $Z(\alpha')X(\beta')$ commute for all $(\alpha,\beta), (\alpha',\beta')$ in a given ray, then
$$
\begin{align}
D(\alpha,\beta) D(\alpha',\beta') - D(\alpha',\beta')D(\alpha,\beta) &=
\Phi(\alpha,\beta) \Phi(\alpha',\beta') Z(\alpha)X(\beta) Z(\alpha')X(\beta') -
\Phi(\alpha,\beta) \Phi(\alpha',\beta') Z(\alpha')X(\beta') Z(\alpha)X(\beta) \\
&= \Phi(\alpha,\beta) \Phi(\alpha',\beta') \left[
Z(\alpha)X(\beta) Z(\alpha')X(\beta') - Z(\alpha')X(\beta') Z(\alpha)X(\beta)
\right] \\
&= 0.
\end{align}
$$

So the phase changes nothing, the sets of commuting operators will remain commuting independently of the phase chosen. But what about their simultaneous set of eigenvectors? Well the set of commuting operators $\{Z(\alpha)X(\mu \alpha)\}$ share a unique eigenbasis $\{|\psi^\mu_\nu\rangle\}$, i.e., 
$$
Z(\alpha)X(\mu \alpha) |\psi^\mu_\nu\rangle = \lambda |\psi^\mu_\nu\rangle,
\quad \forall \alpha \in F.
$$

Now these states are mutually unbiased with respect to other curves parametrized by $\mu$. But recall that the whole Pauli group is of order $2^{2n+1}$, which in this case is 2,048, while the subset parametrized by the phase space is of size $2^{2n}$, which is 1,024. So how does the particular selection of Pauli operators without phase affect the eigenvalues? Let's say $|\psi^\mu_\nu\rangle$ is en eigenvector of $Z(\alpha)X(\mu\alpha)$, then
$$
D(\alpha,\beta) |\psi^\mu_\nu\rangle = \Phi(\alpha,\beta) \lambda |\psi^\mu_\nu\rangle.
$$

So the eigenvalue is affected by the phase of $D(\alpha,\beta)$, how can we identify the $+1$-eigenvector? Especially since in order to construct the set of MUBs we end up actually using a bigger group than the Pauli group, by considering phases $\{1,-1,i,-i\}$ and not just those generated by the group character, i.e., $\{1,-1\}$. It seems that we will end up with eigenvectors whose eigenvalues are not just $+1$ and $-1$. Do we still only consider $+1$-eigenvectors, or should we consider $i$-eigenvectors as well?

Let's see an example. We know that the standard basis is an eigenbasis of the $Z(\alpha)$ operators.

In [237]:
for i in range(d):
    s = identity_matrix(d)[:,i]
    stab = True
    for k in F:
        if Z(k)[:,i] != s:
            stab = False
            break
    if stab:
        print(i, 'is stabilizer')

0 is stabilizer


As we can see, the only stabilizer for the set $\{Z(\alpha)\}$ is the vector $|0\rangle$. The brute force code used here can be used for any MUBs. But we first have to construct the MUBs.

I will now attempt to construct the full set of MUBs for 5 qubits following the physicists way. The main idea is to start with operators corresponding to the horizontal axis or the vertical one, and to define a operator that transforms those operators into a complete set of commuting operators. We associate the verticals lines $\alpha = 0$ with the set $\{X(\beta)\}$ and the horizontal lines $\beta = 0$ with the set $\{Z(\alpha)\}$. The rest of the curves are defined via some bijective function $\beta = f_\mu(\alpha)$ such that $f_{\mu=0}(\alpha) = 0$ for all $\alpha$, i.e., the curves cross the origin. These curves are associated to the sets $\{Z(\alpha)X(f_\mu(\alpha))\}$.

These curves can be constructed from the $Z(\alpha)$ operators via the rotation operators $V_{f_\mu}$ which are diagonal in the Fourier basis (why?):
$$
V_{f_\mu} = \sum_{\kappa} c_{\kappa, f_\mu} |\tilde \kappa\rangle \langle\tilde \kappa|.
$$

The coefficients are restrained such that $c_{0,f_\mu} = 1$ and
$$
V_{f_\mu} Z(\alpha) V_{f_\mu}^\dagger = e^{i \phi(\alpha,f_\mu(\alpha))} Z(\alpha) X(f_\mu(\alpha)).
$$

Given this restriction and the explicit definition of $V_{f_\mu}$, we can observe that:

$$
\begin{align}
V_{f_\mu} Z(\alpha) V_{f_\mu}^\dagger 
&= \left(\sum_{\ell} c_{\ell, f_\mu} |\tilde \ell\rangle \langle\tilde \ell|\right)
Z(\alpha) \left(\sum_{\kappa} c_{\kappa, f_\mu} |\tilde \kappa\rangle \langle\tilde \kappa|\right)^\dagger \\
&= \sum_{\ell,\kappa} c_{\ell, f_\mu} c_{\kappa, f_\mu}^* |\tilde \ell\rangle \langle\tilde \ell| Z(\alpha) |\tilde \kappa\rangle \langle\tilde \kappa| \\
&= \sum_{\ell,\kappa} c_{\ell, f_\mu} c_{\kappa, f_\mu}^* |\tilde \ell\rangle \langle\tilde \ell| \widetilde{\kappa+\alpha}\rangle \langle\tilde \kappa| \\
&= \sum_{\kappa} c_{\kappa + \alpha, f_\mu} c_{\kappa, f_\mu}^* |\widetilde{\kappa+\alpha}\rangle\langle\tilde \kappa|,
\end{align}
$$
and
$$
e^{i \phi(\alpha,f_\mu(\alpha))} Z(\alpha) X(f_\mu(\alpha))
= e^{i \phi(\alpha,f_\mu(\alpha))} Z(\alpha) \sum_{\kappa} \chi(-f_\mu(\alpha) \kappa) |\tilde \kappa\rangle\langle\tilde \kappa|
= e^{i \phi(\alpha,f_\mu(\alpha))} \sum_{\kappa} \chi(-f_\mu(\alpha) \kappa) |\widetilde{\kappa + \alpha}\rangle \langle \tilde\kappa|,
$$
so that
$$
c_{\kappa+\alpha,f_\mu} c_{\kappa,f_\mu}^* = e^{i \phi(\alpha,f_\mu(\alpha))} \chi(-f_\mu(\alpha)\kappa).
$$

And so as long as we can solve this equation, we can find the rotation operators that transform $Z(\alpha)$ into some phase multiple of $Z(\alpha)X(f_\mu(\alpha))$. Consider the case $\kappa = -\alpha$, then on the left hand side we have
$c_{0,f_\mu} c_{-\alpha,f_\mu}^* = c_{-\alpha,f_\mu}^*$, and on the right we have $\chi(f_\mu(\alpha)\alpha))$, so the equation reads:
$$
c_{-\alpha,f_\mu}^* = e^{i\phi(\alpha,f_\mu(\alpha))}\chi(f_\mu(\alpha)\alpha)).
$$

IF $c_{\alpha,f_\mu} = e^{i\phi(\alpha,f_\mu(\alpha))}$, then we obtain
$$
1 = c_{\alpha,f_\mu} c_{-\alpha,f_\mu} \chi(\alpha f_\mu(\alpha)).
$$

According to Sainz, a solution to this equation for odd characteristic is given by $c_{\alpha,f_\mu} = \chi(-2^{-1} \alpha f_\mu(\alpha))$. Let's see (we assume as she did that $f_\mu(-\alpha) = -f_\mu(\alpha)$ always):
$$
\begin{align}
c_{\alpha,f_\mu} c_{-\alpha,f_\mu} \chi(\alpha f_\mu(\alpha))
&= \chi(-2^{-1} \alpha f_\mu(\alpha)) \chi(2^{-1} \alpha f_\mu(-\alpha)) \chi(\alpha f_\mu(\alpha)) \\
&= \chi(-2^{-1} \alpha f_\mu(\alpha)) \chi(-2^{-1} \alpha f_\mu(\alpha)) \chi(\alpha f_\mu(\alpha)) \\
&= \chi(-\alpha f_\mu(\alpha)) \chi(\alpha f_\mu(\alpha)) \\
&= 1.
\end{align}
$$

What is this condition on $f_\mu$? Assume that the mapping is "additive", i.e., $f_\mu(\alpha + \beta) = f_\mu(\alpha) + f_\mu(\beta)$ (notice that presemifield operations satisfy this condition). It follows immediatly if $f_\mu$ is linear of course. Maybe not so trivial...

How about we construct a rotation operator and verify the mathematical claims? Except for characteristic two it is not so trivial!

According to Sainz the coefficients are now determined by the equation
$$
\Phi(\alpha,f_\mu(\alpha))^2 = c_{\alpha,f_\mu}^2 = \chi[-\alpha \beta].
$$
She states that a solution can be found by lifting the operations to the Teichmüller
$$
\Phi(\alpha,f_\mu(\alpha)) = c_{\alpha,f_\mu} = \omega(\hat \alpha f_{\hat \mu}(\hat \alpha)),
$$
where $\omega(\hat \alpha) = i^{Tr(\hat \alpha)}$ but the trace is taken in the Galois ring.

In [241]:
# Must be in power order
def Lift(a):
    return T[F.index(a)]

In [442]:
# Sainz's phase for Wootters' curves
def Phi(u,v):
    uhat = Lift(u)
    vhat = Lift(v)
    a, b = dumb2adic(uhat * (uhat * vhat))
    pwr  = lift(ringTrace(a, b))
    return I**Integer(pwr)

In [447]:
def D(u,v):
    phase = Phi(u,v)
    # phase = 1
    return phase * Z(u) * X(v)

First how do we obtain the Fourier basis projections $|\tilde \kappa\rangle\langle\tilde \kappa|$?

### Finding stabilizer states using Kanat's construcction.

We know that a vector $v$ is a stabilizer state if there is a subset $\mathcal S$ of the Pauli group with $|\mathcal S| = d$ such that $v$ is a simultaneous $+1$-eigenvector of all of the elements of $\mathcal S$.

We know that each totally isotropic subspace of $F \oplus F$ gives us a complete set of commuting Pauli operators, and so a complete set of simultaneous eigenvectors.

In Kanat's paper we have that the vector
$$d_{m,v} = \sum_{w \in F} \omega^{Tr(\hat w (\hat w \circ \hat m) + 2 \hat w \hat v)} e_w$$
is an eigenvector of the operator $D_{u,u\circ m} = X_u Z_{u \circ m}$ with eigenvalue
$$\omega^{Tr(2 \hat u \hat v - \hat u (\hat u \circ \hat m))}.$$

So an eigenvector $d_{m,v}$ is a $+1$-eigenvector of $D_{u,u\circ m}$ if
$$Tr(2 \hat u \hat v - \hat u (\hat u \circ \hat m)) = 0.$$ Therefore $d_{m,v}$ is a stabilizer state if this last equation is satisfied *for all* $u \in F$.

In [62]:
def Kantor(w, m):
    return m**2*w + m*ringTrace(w, 0) + ringTrace(m*w, 0)

In [74]:
for m in T:
    for i, v in enumerate(T):
        s = True
        for j, u in enumerate(T):
            # a, b = dumb2adic(GR(2) * u * v - u * (Kantor(u, m)))
            a, b = dumb2adic(GR(2) * u * v - u * (u * m))
            if ringTrace(a, b) != GR(0):
                s = False
                break
        if s == True:
            print(m,v)

0 0


No luck trying this, let's create the standard MUBs for 5 qubits and see if we're lucky by using Sainz phase.

In [259]:
def createMubs(k=None):
    mubs = [identity_matrix(d)]
    for m in T[:k]:
        Bm = zero_matrix(SR, d, d)
        for j, v in enumerate(T):
            for i, w in enumerate(T):
                # wom = m**2*w + m*ringTrace(w, 0) + ringTrace(m*w, 0)
                wom = m * w
                a, b = dumb2adic(w * wom + GR(2) * w * v)
                pwr  = Integer(lift(ringTrace(a, b)))
                Bm[i,j] = 1/sqrt(d) * I**pwr
        mubs.append(Bm)
    return mubs

In [260]:
mubs = createMubs()

By definition, the phase for the vertical line operators $X(\beta)$ is equal to 1. A quick glance shows that the only vector from the Fourier basis that is a stabilizer is the first column:

In [330]:
for i in range(1, d):
    m = mubs[1] - X(F[i]) * mubs[1]
    if m[:,0] != zero_matrix(d,1):
        raise Exception('Not stabilizer.')

So we have two stabilizers so far from the standard MUBs. Let's check the diagonal.

In [493]:
(mubs[6].conjugate_transpose() * mubs[14]).apply_map(lambda t: abs(t)**2) == ones_matrix(d,d)/32

True

In [508]:
((D(F[10],F[10]) * mubs[2][:,10]) / mubs[2][:,10]).diagonal()

[I,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]